In [1]:
#H.7.1
def householder(state):
    return np.eye(state.shape[0])- 2*np.outer(state,state)
    pass

In [ ]:
#H.7.2a
k_bits = 2
n_bits = 2
all_bits = k_bits + n_bits
aux = range(k_bits)
main = range(k_bits, all_bits)
dev = qml.device("default.qubit", wires=all_bits)

def PREPARE(alpha_list):
    zero_vec = np.array([1] + [0]*(2**k_bits - 1))
    alpha_list = np.sqrt(alpha_list) / np.linalg.norm(alpha_list)
    return householder((zero_vec - alpha_list) / np.linalg.norm(zero_vec -alpha_list))

In [ ]:
#H.7.2b
def SELECT(U_list):
    for index in range(2**k_bits):
        ctrl_str = np.binary_repr(index, k_bits)
        qml.ControlledQubitUnitary(U_list[index], control_wires=aux, 
                                   wires=main, control_values=ctrl_str)

def LCU(alpha_list, U_list):
    qml.QubitUnitary(PREPARE(alpha_list), wires=aux)
    SELECT(U_list)
    qml.adjoint(qml.QubitUnitary)(PREPARE(alpha_list), wires=aux)

In [ ]:
#H.7.2c
U_list = [np.kron(qml.PauliX.compute_matrix(), qml.PauliX.compute_matrix()),
          np.kron(qml.PauliZ.compute_matrix(), qml.PauliZ.compute_matrix()),
          np.kron(qml.PauliX.compute_matrix(), qml.PauliZ.compute_matrix()),
          np.kron(qml.PauliZ.compute_matrix(), qml.PauliX.compute_matrix())]
alpha_list = [1, 0.5, 0.5, 1]

@qml.qnode(dev)
def my_circuit():
    for _ in main:
	    qml.PauliX(wires=_)
    LCU(alpha_list, U_list)
    return qml.state()

print("The amplitudes on the main register are proportional to", my_circuit()[:4], ".")

In [ ]:
#H.7.3
@qml.qnode(dev)
def quantum_memory(beta_list):
    U_list = [np.identity(4), np.identity(4), 
              np.identity(11), np.identity(11)]
    bell = np.matmul(np.kron(qml.Hadamard.compute_matrix(), np.eye(2)),qml.CNOT.compute_matrix())
    U_list = [np.kron(qml.PauliZ.compute_matrix(), qml.PauliZ.compute_matrix()),
    np.kron(qml.PauliZ.compute_matrix(),qml.PauliX.compute_matrix()),
    np.kron(qml.PauliX.compute_matrix(),qml.PauliZ.compute_matrix()),
    np.kron(qml.PauliX.compute_matrix(),qml.PauliX.compute_matrix())]
    beta_sq = np.square(beta_list)
    qml.QubitUnitary(PREPARE(beta_sq), wires=aux)
    SELECT(U_list)
    return qml.state()

beta_list = [1, 0, 0, 1]
normalized_coefficients = [quantum_memory(beta_list)[i].item() for i in range(0, 20, 5)]
print("The amplitudes on the main register are proportional to", normalized_coefficients, ".")